# Introduction

![image](./imgs/‎GenAIEnterprises.‎009.png)

![image](./imgs/‎GenAIEnterprises.‎010.png)

![image](./imgs/‎GenAIEnterprises.‎011.png)


In [3]:
import boto3
import os
from IPython.display import display, Markdown
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

def get_api_key(ssm_client, parameter_path):
    '''Get the OpenAI API key from the SSM Parameter Store'''
    try:
        response = ssm_client.get_parameter(
            Name=parameter_path,
            WithDecryption=True
        )
        return response['Parameter']['Value']
    except ssm_client.exceptions.ParameterNotFound:
        raise Exception(f'Parameter {parameter_path} not found in SSM Parameter Store')

# Create an SSM client using Boto3
region_name = os.getenv('AWS_REGION', 'us-east-1') 
ssm = boto3.client('ssm', region_name=region_name)

openai_api_key = get_api_key(ssm_client=ssm, parameter_path='/openai/api_key')
langchain_api_key = get_api_key(ssm_client=ssm, parameter_path='/langchain/api_key')


os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key




# Set the model variable based on the current date
llm_model = "gpt-3.5-turbo-16k"

# Create the vector store and embedding function
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory='docs/chroma/',
    embedding_function=embedding
)

In [4]:
import langsmith

import nest_asyncio
nest_asyncio.apply()

from langchain import chat_models, smith

# Replace with the chat model you want to test
my_llm = chat_models.ChatOpenAI(temperature=0)

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa",
    ],
    custom_evaluators=[],
    eval_llm=chat_models.ChatOpenAI(temperature=0)
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="ds-stupendous-chair-30",
    llm_or_chain_factory=my_llm,
    evaluation=eval_config,
    project_name="test-artistic-trousers-61",
    verbose=True,
)

View the evaluation results for project 'test-artistic-trousers-61' at:
https://smith.langchain.com/o/f09d95d4-78e8-5dcb-a136-09de2b589be4/projects/p/65ecd9d2-2b91-4a59-b303-3191587ed253
[------------------------------------------------->] 6/6
 Eval quantiles:
                         0.25  0.50  0.75
COT Contextual Accuracy  0.25   1.0   1.0
